# Скрипт для автоматизации сборки ежегодного денрожденного анонса студии Артемия Лебедева

Пример анонса: https://www.artlebedev.ru/news/2023/party/

In [726]:
import pandas as pd
import math
import json
import requests
import gspread
from transliterate import translit, slugify
from re import search, match, sub
from typograf import RemoteTypograf
import time
import xmltodict

## Загрузка данных

Необходимо предварительно создать сервисный аккаунт гугл и дать ему права на чтение таблицы. 
Подробнее: https://cloud.google.com/iam/docs/service-account-overview?_ga=2.63357484.-2024291479.1695986852

В моем случае ключи к сервисному аккаунту хранятся в файле als-28-birthday-bot-8f455c2a03f2.json.
Айди таблицы с подрядчиками — 1IAxjV8BwAriO5FOop4JkJ1tuHJrZJeQYghCxJxZ8t2g (берется из урла)

In [745]:
gc = gspread.service_account(filename='als-28-birthday-bot-8f455c2a03f2.json')

worksheet = gc.open_by_key("1IAxjV8BwAriO5FOop4JkJ1tuHJrZJeQYghCxJxZ8t2g")

In [746]:
df = pd.DataFrame(worksheet.get_worksheet(1).get_all_records())

# Можно так же скачать эксель руками и читать из него (не рекомендуется)
# df = pd.read_excel('data.xlsx', sheet_name=1) 

## Подготовка данных

Оставляем нужные колонки

In [747]:
df = df[['Наименование','Описание', 'Контакты', 'Категория для анонса', 'ОТЗЫВ (для анонса)']]

df.head(10)

,Наименование,Описание,Контакты,Категория для анонса,ОТЗЫВ (для анонса)
0,Эко-плант,Крупнейший оптовый и розничный поставщик дерев...,https://ecoplant-pitomnik.ru/,Оформление,Сделали джунгли!
1,Жан-плов китчен,Шеф-повар Мансур Холматов готовит ташкентский ...,+7 925 760-50-30\ntiktok.com/@jeanplove,Еда,Очень вкусно!
2,Фермер Зуфар,Зуфар работает в России с мыслями о шумном Рио...,@unforgettablefarm,Еда,На редкость нежное мясное блюдо. Целое блюдо
3,ХукаПлейс,Родоначальники кальянной индустрии в России со...,hookahplace.ru\ninstagram.com/hookahplaceoffic...,развлечения,Лучшие из всех! Постоянно меняли кальяны гостя...
4,Сыры «Белое озеро»,Молокозавод в Оренбурге производит отличные сы...,vk.com/sssruslan,Еда,"Очень много и вкусно, особенно с мёдом"
5,Мясо и мед «Ашкадарский»,В 42 км от ближайших поселений заслуженный пче...,@khalitoff,Еда,
6,Фитофит,"Производитель иван-чая, травяных чаев и цукато...",+7 931 501-25-50 \nvolteaman.ru\nvk.com/fitofit,Напитки,
7,Бибиколь,Дистрибьюторы уникальной детской смеси «Нэнни»...,8 800 200-888-0\nbibicall.ru \nt.me/bibicallru...,Напитки,Алкогольные коктейли на основе детского питани...
8,Кафе «Сатива»,Предлагают клиентам продукты из технической (б...,+7 911 501-09-11\nsativa.taplink.ws\ninstagram...,Еда,
9,Московская пивоваренная компания,Российский производитель и дистрибьютор пива и...,+7 495 788-54-33\nmosbrew.ru\nt.me/mosbrew\n,Напитки,


Переименовываем колонки, удаляем лишние строки

In [748]:
df = df.rename(columns={'Категория для анонса': 'Категория', 'ОТЗЫВ (для анонса)': 'Отзыв'})
df = df.drop(df[df['Наименование'] == ''].index)
df = df.drop(df[df['Категория'] == ''].index)
df = df[df['Наименование'] != 'Филонов']

Уникальные значения категорий. Замечаем опечатки.

In [750]:
df['Категория'].unique()

array(['Оформление', 'Еда', 'развлечения', 'Напитки', 'Красота',
       'Развлечения', 'еда', 'напитки', 'напикти', 'Безопасность',
       'артисты', 'оформление', 'съемка', 'Артисты', 'Артситы'],
      dtype=object)

Функция для замены значений на названия из словаря

In [752]:
def replace_value(x, values):
    for key, value in values.items():
        if (x == key):
            return value

Заменяем названия категорий

In [753]:
replacements = {
    "Еда": "food",
    "еда": "food",
    "Напитки": "drink",
    "напитки": "drink",    
    "напикти": "drink",    
    "Безопасность": "security",
    "Оформление": "interior",
    "оформление": "interior",
    "Технологии": "tech",
    "технологии": "tech",
    "Развлечения": "fun",
    "развлечения": "fun",
    "артисты": "artist",
    "съемка": "video",
    "Артисты": "artist",
    "Артситы": "artist",
    "Красота": "beauty"
}

df['Категория'] = df['Категория'].apply(lambda x: replace_value(x, replacements))

Новые значения категорий

In [754]:
df['Категория'].unique()

array(['interior', 'food', 'fun', 'drink', 'beauty', 'security', 'artist',
       'video'], dtype=object)

## Парсинг данных

Здесь мы собираем контакты, типографируем тексты, добавляем в таблицу соответствующие поля

In [755]:
rt = RemoteTypograf()

df_wip = df.copy()

for index, row in df_wip.iterrows():
    descr = ''
    descrFull = ''
    contacts = [x.strip() for x in row[2].split('\n')]
    instagram = ''
    fb = ''
    vk = ''
    telegram = ''
    tel = ''
    tel2 = ''
    site = ''
    site2 = ''
    mail = ''
    tiktok = ''
    youtube = ''
    
    for item in contacts:
        if item == '':
            continue
            
        if 'instagram' in item:
            instagram = item.replace('instagram.com', '').replace('https://instagram.com', '').replace('/', '')
            continue    
        
        if 'vk.com/' in item:
            vk = item.replace('vk.com', '').replace('https://vk.com', '').replace('/', '')
            continue
            
        if 'youtube.com' in item:
            youtube = item.replace('youtube.com', '').replace('https://', '').replace('www.', '').replace('/', '')
            continue
            
        if 'facebook.com' in item:
            fb = item.replace('facebook.com', '').replace('https://', '').replace('www.', '').replace('/', '')
            continue
        
        if search('t.me/', item) or item[0] == '@':
            telegram = item.replace('t.me', '').replace('https://t.me', '').replace('/', '').replace('@', '')
            continue
        
        if search(r'\+7', item) or item.startswith('8 8') or item.startswith('8 9'):
            value = item.replace(' ', '').replace('-', '').replace('—', '').replace(',', '')
            value = sub(r'[а-яА-Я]+', '', value)
            value = sub(r'^8', '+7', value)
            
            if tel == '':
                tel = value
            elif tel2 == '':
                tel2 = value
            continue

        if search('tiktok.com/@', item):
            tiktok = item.replace('tiktok.com/@', '')
            continue
            
        if search('@', item) and item[0] != '@':
            mail = item
            continue
            
        value = 'https://' + item.replace('https://', '')
            
        if site == '':
            site = value
        elif site2 == '':
            site2 = value
    
    text = [x for x in rt.try_process_text(row[1].strip()).split('\n')]
    time.sleep(1.01)
    
    descr = text[0]
    descrFull = '\n'.join(text)
        
    tema = ''
    
    if row[4]:
        tema = rt.try_process_text(row[4])
        time.sleep(1.01)
    
    df_wip.loc[index, 'descr'] = descr
    df_wip.loc[index, 'descrFull'] = descrFull
    df_wip.loc[index, 'site'] = site
    df_wip.loc[index, 'site2'] = site2
    df_wip.loc[index, 'instagram'] = instagram
    df_wip.loc[index, 'tel'] = tel
    df_wip.loc[index, 'tel2'] = tel2
    df_wip.loc[index, 'telegram'] = telegram
    df_wip.loc[index, 'vk'] = vk
    df_wip.loc[index, 'tema'] = tema
    df_wip.loc[index, 'mail'] = mail
    df_wip.loc[index, 'tiktok'] = tiktok
    df_wip.loc[index, 'fb'] = fb
    
    print(row['Наименование'])


Эко-плант
Жан-плов китчен
Фермер Зуфар
ХукаПлейс
Сыры «Белое озеро»
Мясо и мед «Ашкадарский»
Фитофит
Бибиколь
Кафе «Сатива»
Московская пивоваренная компания
Cтепь и ветер
Глиттер тингз бьюти
Клиника косметологии «ЛуЛу мед»
Выставка «Цветомузыка СССР»
Нейроботикс
Звукотерапевт Надежда Маслова
Ма.хани
Вероника, тату-мастер
Алкогольный бар «Сайленцио»
Брио айс
Природная вода «Воргол»
Щелкунчик
Брю бай крю
Винодельня «Жаков»
АлёнаБьёт
Лазеркат
Кинки-пати
Зеро пойнт
Бар «Лью»
Простые правила
Анастасия Белова, мастер мехенди
Ямайка
Александра, художник-шаржист
Дабл дабл
Блинбери
Раш попкорн
Ниндзя гудс
Лето в банке
ЙУХ (Йорик Убил Хамлета)
Такамака
Дринксам
Фейсрум
Дагфуд
Игра го
Студия 3D
Пян-се
Чаплинс пицца
Бар «Явка»
Вятич
Мастерская шоколада «Вокачо»
Ипполит
Гули Були
Шато Пино
Би джин
Водка «Финист»
Барристер
Царская
Вейн
Пасс поинт
Кардбист
Пинкит
Охранная группа «Оберегъ»
Гет энерджи
Цифроград
1851
Пранна
Хоум оунер
Карина Лобанова 
Сергей Фонвизин, фотограф
Максим Колибердин
Анастас

Обновленная таблица

In [756]:
df_wip.head(5)

,Наименование,Описание,Контакты,Категория,Отзыв,descr,descrFull,site,site2,instagram,tel,tel2,telegram,vk,tema,mail,tiktok,fb
0,Эко-плант,Крупнейший оптовый и розничный поставщик дерев...,https://ecoplant-pitomnik.ru/,interior,Сделали джунгли!,Крупнейший оптовый и&nbsp;розничный поставщик ...,Крупнейший оптовый и&nbsp;розничный поставщик ...,https://ecoplant-pitomnik.ru/,,,,,,,Сделали джунгли!\n,,,
1,Жан-плов китчен,Шеф-повар Мансур Холматов готовит ташкентский ...,+7 925 760-50-30\ntiktok.com/@jeanplove,food,Очень вкусно!,Шеф-повар Мансур Холматов готовит ташкентский ...,Шеф-повар Мансур Холматов готовит ташкентский ...,,,,+79257605030,,,,Очень вкусно!\n,,jeanplove,
2,Фермер Зуфар,Зуфар работает в России с мыслями о шумном Рио...,@unforgettablefarm,food,На редкость нежное мясное блюдо. Целое блюдо,Зуфар работает в&nbsp;России с&nbsp;мыслями о&...,Зуфар работает в&nbsp;России с&nbsp;мыслями о&...,,,,,,unforgettablefarm,,На&nbsp;редкость нежное мясное блюдо. Целое бл...,,,
3,ХукаПлейс,Родоначальники кальянной индустрии в России со...,hookahplace.ru\ninstagram.com/hookahplaceoffic...,fun,Лучшие из всех! Постоянно меняли кальяны гостя...,Родоначальники кальянной индустрии в&nbsp;Росс...,Родоначальники кальянной индустрии в&nbsp;Росс...,https://hookahplace.ru,,hookahplaceofficial,,,hookahplacetv,,Лучшие из&nbsp;всех! Постоянно меняли кальяны ...,,,
4,Сыры «Белое озеро»,Молокозавод в Оренбурге производит отличные сы...,vk.com/sssruslan,food,"Очень много и вкусно, особенно с мёдом",Молокозавод в&nbsp;Оренбурге производит отличн...,Молокозавод в&nbsp;Оренбурге производит отличн...,,,,,,,sssruslan,"Очень много и&nbsp;вкусно, особенно с&nbsp;мёд...",,,


Переименовываем колонки, сбрасываем индексы

In [757]:
df_wip = df_wip.rename(columns={'Наименование': 'name'})
df_wip = df_wip.rename(columns={'Категория': 'category'})
df_wip = df_wip.reset_index(drop=True)

Расставляем категории в правильном порядке, снова сбрасываем индексы

In [758]:
df_final = df_wip.copy()

df_final = pd.concat([
    df_final.loc[df_final["category"] == "fun"],
    df_final.loc[df_final["category"] == "beauty"],
    df_final.loc[df_final["category"] == "artist"],
    df_final.loc[df_final["category"] == "drink"],
    df_final.loc[df_final["category"] == "food"],
    df_final.loc[df_final["category"] == "interior"],
    df_final.loc[df_final["category"] == "security"],
    df_final.loc[df_final["category"] == "video"]
])

df_final = df_final.reset_index(drop=True)

### Названия и фотографии

Кривые и длинные названия заменяем на значения из словаря.
Фиксируем порядок подрядчиков через поля next и prev.
Проверяем наличие фото по прямой ссылке вида `https://www.artlebedev.ru/news/2023/party/suppliers/{suplier_name}/photo_{n}.jpg`. Здесь сразу можно увидеть каких фото не хватает, где нарушена нумерация. 

<b>Важно:</b> Все папки с фотографияеми должны соответсоввать латинским названиям подрядчиков, названия фотографий должны иметь вид `photo_{n}.jpg`, в нумерации не должно быть пропусков.

In [759]:
suppliers_url = "https://www.artlebedev.ru/news/2023/party/suppliers/"
suppliers = {}
replacements = {
    "chaplins-pitstsa": "chaplins-pizza",
    "pjan-se-korejskaja-parovaja-bulochka-s-nachinkoj": "pjan-se",
    "antoha-ms": "antoha-mc",
    "radio-kunst": "radio-kunst",
    "tima-akimov": "tima-akimov",
    "kostja-sidorkov": "kostja-sidorkov",
    "mjaso-i-med-ashkadarskij": "ashkadarskij",
    "sedobnye-stakanchiki-drink-en-it": "drink-en-it",
    "moskovskaja-pivovarennaja-kompanija": "mpk",
    "klinika-kosmetologii-lulu-med": "lulu-med",
    "nadezhda-bessonova-tarolog": "nadezhda-bessonova",
    "zvukoterapevt-nadezhda-maslova": "nadezhda-maslova",
    "alkogolnyj-bar-sajlentsio": "sajlentsio",
    "prirodnaja-voda-vorgol": "vorgol",
    "elena-ushakova-spetsialist-po-mak-kartam": "elena-ushakova",
    "vsevolod-latunov-illjuzionist": "vsevolod-latunov",
    "bezalkogolnyj-i-alkogolnyj-bar-lju": "lju",
    "anastasija-belova-master-mehendi": "anastasija-belova",
    "aleksandra-hudozhnik-sharzhist": "aleksandra-hudozhnik-sharzhist",
    "juh-jorik-ubil-hamleta": "juh",
    "fotobudka-sejchas-vyletit-ptichka": "fotobudka",
    "grigorij-nevarko-fokusy-s-kartami": "grigorij-nevarko",
    "tatjana-zueva-tarolog": "tatjana-zueva",
    "vystavka-tsvetomuzyka-sssr": "tsvetomuzyka",
    "masterskaja-shokolada-vokacho": "vokacho",
    "bibikol-detskoe-pitanie": "bibikol",
    "bibikol-ne-detskoe-pitanie": "bibikol-ne-detskoe",
    "hukaplejs": "hookaplace",
    "neyrobotiks": "neurobotics",
    "alenabet": "alyonabyot",
    "kinki-pati": "kinky-party",
    "darya-kovalskaya-vizazhist-brovist": "darya-kovalskaya",
    "prosekko-shou": "proseko-show",
    "feysrum": "faceroom",
    "glitter-tingz-byuti": "glitter-tingz-beauty",
    "get-enerdzhi": "get-energy",
    "lazerkat": "lasercut",
    "zero-poynt": "zero-point",
    "tstep-i-veter": "step-i-veter",
    "saylentsio": "silencio",
    "drinksam": "drinksome",
    "bryu-bay-kryu": "brew-by-crew",
    "chiz-it-beykeri": "cheese-it-bakery",
    "brio-ays": "brio-ice",
    "dabl-dabl": "double-double",
    "rash-popkorn": "rush-popcorn",
    "darya-kovalskaya-vizazhist-brovist": "darya-kovalskaya",
    "ohrannaya-gruppa-obereg": "obereg",
    "sergej-fonvizin-fotograf": "sergey-fonvizin",
    "real-medzhik": "real-magic",
    "kardbist": "cardbeast",
    "houm-ouner": "home-owner",
    "nindzya-guds": "ninja-goods",
    "dagfud": "dagfood",
    "ohrannaya-gruppa-obereg": "obereg",
    "sedobnye-stakanchiki-dlya-napitkov-i-produktov-pitaniya-drink-end-it": "drink-and-it",
    "drink-end-it": "drink-and-it",
    "tsifrograd": "cyfrograd"
}

for index, row in df_final.iterrows():
    if str(row[0]).isnumeric():
        slug = str(row[0])
    else:
        slug = slugify(translit(row[0], "ru"))
        
    if slug in replacements.keys():
        slug = replace_value(str(slug), replacements)
        
    slug = slug.replace('j', 'y')
    
    if slug in replacements.keys():
        slug = replace_value(str(slug), replacements)
    
    df_final.loc[index, 'slug'] = slug
    
    if index == 0:
        df_final.loc[len(df_final.index) - 1, 'next'] = slug
    else:
        df_final.loc[index - 1, 'next'] = slug

    if index == len(df_final.index) - 1:
        df_final.loc[0, 'prev'] = slug
    else:
        df_final.loc[index + 1, 'prev'] = slug
        
    counter = 0
        
    while 1:    
        image_url = suppliers_url + slug + "/photo_" + str(counter + 1) + ".jpg"

        try:
            r = requests.get(image_url)

            r.raise_for_status()
            
            counter += 1
        except requests.exceptions.HTTPError as error:
            break
    
    df_final.loc[index, 'photos'] = int(counter)
    
    print(f'{slug} — {counter}')

hookaplace — 10
tsvetomuzyka — 9
neurobotics — 8
nadezhda-maslova — 5
lasercut — 4
kinky-party — 10
prostye-pravila — 10
yamayka — 6
aleksandra-hudozhnik-sharzhist — 5
igra-go — 4
studiya-3d — 8
get-energy — 3
pranna — 10
proseko-show — 5
nadezhda-bessonova — 3
vsevolod-latunov — 3
elena-ushakova — 3
alena-runolog — 6
tatyana-zueva — 2
kristina-hiromant — 0
grigoriy-nevarko — 1
glitter-tingz-beauty — 4
lulu-med — 5
veronika-tatu-master — 4
alyonabyot — 5
anastasiya-belova — 5
faceroom — 4
darya-kovalskaya — 6
1851 — 7
mashkov — 3
radio-kunst — 9
antoha-mc — 10
tima-akimov — 8
kostya-sidorkov — 5
fitofit — 5
bibikol — 9
mpk — 6
step-i-veter — 9
silencio — 5
vorgol — 5
brew-by-crew — 6
vinodelnya-zhakov — 6
zero-point — 10
bar-lyu — 6
leto-v-banke — 5
yuh — 8
takamaka — 9
drinksome — 10
bar-yavka — 5
vyatich — 5
ippolit — 9
guli-buli — 10
shato-pino — 10
bi-dzhin — 5
vodka-finist — 9
barrister — 10
tsarskaya — 5
veyn — 5
zhan-plov-kitchen — 10
fermer-zufar — 5
syry-beloe-ozero — 4
ashkad

## Выгрузка конфигурационных файлов

### config.p
Здесь мы собираем всю информацию о карточках подрядчиков (поле parnersList в config.p). Вывод можно скопировать и вставить туда.

In [760]:
result = ''

prev_category = ''

def parser_shield(value):
    return str(value).rstrip('\r\n').replace('Высококвалифицированные', 'Высоко&shy;квалифи&shy;цирован&shy;ные').replace(';', '^;').rstrip()

for index, row in df_final.iterrows():
    category = row['category']
    
    if category != prev_category:
        result += f'''
        # {category}
        '''

    prev_category = category
    
    name = parser_shield(row['name'])
    tema = parser_shield(row['tema'])
    descr = parser_shield(row['descr'])
    descrFull = parser_shield(row['descrFull'])
    mail = parser_shield(row['mail'])
    photos = row['photos']
    
    if photos == '0':
        photos = ''
    else:
        photos = str(photos)[:-2]
    
    result += f'''
    $.[{row['slug']}][
        $.category[{category}]
        $.next[{row['next']}]
        $.prev[{row['prev']}]
        $.name[{name}]
        $.tema[{tema}]
        $.descr[{descr}]
        $.descrFull[{descrFull}]
        $.site[{row['site']}]
        $.site2[{row['site2']}]
        $.instagram[{row['instagram']}]
        $.fb[{row['fb']}]        
        $.vk[{row['vk']}]
        $.tel[{row['tel']}]
        $.tel2[{row['tel2']}]
        $.telegram[{row['telegram']}]
        $.tiktok[{row['tiktok']}]
        $.mail[{mail}]
        $.photos[{photos}]
    ]
    '''

with open("partnerList.txt", "w") as text_file:
    text_file.write(result)

print(result)


        # fun
        
    $.[hookaplace][
        $.category[fun]
        $.next[tsvetomuzyka]
        $.prev[andrey-semenov]
        $.name[ХукаПлейс]
        $.tema[Лучшие из&nbsp^;всех! Постоянно меняли кальяны гостям, вкусный табак и&nbsp^;ребята классные]
        $.descr[Родоначальники кальянной индустрии в&nbsp^;России создали крупнейшую франшизную сеть кальянных заведений в&nbsp^;мире. Более двух сотен заведений работают в&nbsp^;21&nbsp^;стране мира.]
        $.descrFull[Родоначальники кальянной индустрии в&nbsp^;России создали крупнейшую франшизную сеть кальянных заведений в&nbsp^;мире. Более двух сотен заведений работают в&nbsp^;21&nbsp^;стране мира.]
        $.site[https://hookahplace.ru]
        $.site2[]
        $.instagram[hookahplaceofficial]
        $.fb[]        
        $.vk[]
        $.tel[]
        $.tel2[]
        $.telegram[hookahplacetv]
        $.tiktok[]
        $.mail[]
        $.photos[10]
    ]
    
    $.[tsvetomuzyka][
        $.category[fun]
        $.ne

### index.html

In [761]:
prev_category = ''

for index, row in df_final.iterrows():
    category = row['category']
    slug = row['slug']
    
    if category != prev_category:
        print('\n\n' + category + '\n')

    prev_category = category
    
    print(f'^print_partner_item[$partnerList.{slug}]')



fun

^print_partner_item[$partnerList.hookaplace]
^print_partner_item[$partnerList.tsvetomuzyka]
^print_partner_item[$partnerList.neurobotics]
^print_partner_item[$partnerList.nadezhda-maslova]
^print_partner_item[$partnerList.lasercut]
^print_partner_item[$partnerList.kinky-party]
^print_partner_item[$partnerList.prostye-pravila]
^print_partner_item[$partnerList.yamayka]
^print_partner_item[$partnerList.aleksandra-hudozhnik-sharzhist]
^print_partner_item[$partnerList.igra-go]
^print_partner_item[$partnerList.studiya-3d]
^print_partner_item[$partnerList.get-energy]
^print_partner_item[$partnerList.pranna]
^print_partner_item[$partnerList.proseko-show]
^print_partner_item[$partnerList.nadezhda-bessonova]
^print_partner_item[$partnerList.vsevolod-latunov]
^print_partner_item[$partnerList.elena-ushakova]
^print_partner_item[$partnerList.alena-runolog]
^print_partner_item[$partnerList.tatyana-zueva]
^print_partner_item[$partnerList.kristina-hiromant]
^print_partner_item[$partnerList.grig

### Список всех латинских названий подрячдиков по категориям для фотографа и дизайнера
Необходимо проверить нумерацию фото и отсутсвие пробелов в конце названия папок

In [765]:
prev_category = ''

for index, row in df_final.iterrows():
    category = row['category']
    slug = row['slug']
    
    if category != prev_category:
        print('\n\n#' + category + '\n')

    prev_category = category
    
    print(slug)



#fun

hookaplace
tsvetomuzyka
neurobotics
nadezhda-maslova
lasercut
kinky-party
prostye-pravila
yamayka
aleksandra-hudozhnik-sharzhist
igra-go
studiya-3d
get-energy
pranna
proseko-show
nadezhda-bessonova
vsevolod-latunov
elena-ushakova
alena-runolog
tatyana-zueva
kristina-hiromant
grigoriy-nevarko


#beauty

glitter-tingz-beauty
lulu-med
veronika-tatu-master
alyonabyot
anastasiya-belova
faceroom
darya-kovalskaya


#artist

1851
mashkov
radio-kunst
antoha-mc
tima-akimov
kostya-sidorkov


#drink

fitofit
bibikol
mpk
step-i-veter
silencio
vorgol
brew-by-crew
vinodelnya-zhakov
zero-point
bar-lyu
leto-v-banke
yuh
takamaka
drinksome
bar-yavka
vyatich
ippolit
guli-buli
shato-pino
bi-dzhin
vodka-finist
barrister
tsarskaya
veyn


#food

zhan-plov-kitchen
fermer-zufar
syry-beloe-ozero
ashkadarskiy
kafe-sativa
mahani
brio-ice
schelkunchik
double-double
blinberi
rush-popcorn
ninja-goods
dagfood
pyan-se
chaplins-pizza
vokacho
cheese-it-bakery
katya-belchik
drink-and-it


#interior

eko-plant
card